# Data pre-processing - version Minsky (PPC64LE), multi-GPU, high RAM

## Check hardware
This code segment ensures you are using GPU and high-memory for the analysis.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

## Install PyTorch for CUDA 10.2
This section of the code is required for Minsky as it has 2 GPU. First we uninstall PyTorch as we don't want to use the most recent version.

In [ ]:
# the following needs to be run in the CLI. It doesn't work from the magic here because of multiple "y" prompts
# conda install pytorch
# conda install -c conda-forge notebook
# conda install -c conda-forge jupyter_contrib_nbextensions
# conda install -c conda-forge jupyter_nbextensions_configurator

In [ ]:
!cat /proc/driver/nvidia/version

In [ ]:
# Check that PyTorch sees it
import torch
#torch.zeros(1).cuda()
torch.cuda.is_available()

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
!df -h

In [ ]:
!cat /proc/meminfo

In [ ]:
import gc
gc.collect()

We install dask, which is a memory-friendly version of Pandas. It's a bit more difficult to use, and there's insufficient documentation out there. But nonetheless, needed for machines with lower RAM (less than 120G).

## Uncompress/compress original files
The files come from source with compression in gzip. This code can be used to expand the files by using the Linux command line.

In [ ]:
!gunzip /home/edo/Documents/Security/Data/auth.txt.gz

In [ ]:
!gzip /home/edo/Documents/Security/Data/auth.txt

## Create:  auth.txt >> authUsers.txt
This code combines two datasets: the authentications and the red team one, so we have a single dataset that identifies the authentications that were performed during red team activity (the insider threat). The result is 341MM of events, with 739 labeled as threats.

In [ ]:
import pandas as pd

In [ ]:
authdf = pd.read_csv('/home/edo/Documents/Data/auth.txt',header=None)

The following cell filters so the authentications dataset only has users, not computers. Then it takes the filtered data and exports it into a single file.

In [ ]:
authUsersdf=authdf[authdf[1].str.startswith('U')]
authUsersdf=authUsersdf.rename(columns={0:'unixSecond', 1:'sourceUser', 2:'destinationUser', 3:'sourceComputer', 4:'destinationComputer',5:'authenticationType',6:'logonType', 7:'orientation', 8:'successOrFailure'})
authUsersdf.head()

The following line to save an intermediate file with only user authentications. If the file **authUsers.txt** already exists, this task is not mandatory, and takes around 45 mins.

In [ ]:
authUsersdf.to_csv('/home/edo/Documents/Data/authUsers.txt', single_file = True)

Counts the number of records, or around 341MM for user authentications.

In [ ]:
!wc -l /home/edo/Documents/Data/authUsers.txt

## Create: authUsersRedTeam.txt >> authSentences

In [ ]:
authUsersdf = pd.read_csv('/home/edo/Documents/Data/authUsers.txt')
authUsersdf.head()

In [ ]:
authUsersdf['hourOfDay'] = (authUsersdf['unixSecond'] % 86400 // 3600) + 1

In [ ]:
authUsersdf['dayOfWeek'] = (authUsersdf['unixSecond'] % 604800 // 86400) + 1

In [ ]:
day7=authUsersdf[authUsersdf['dayOfWeek']==7]
day7.head()

In [ ]:
authUsersdf['sentence'] = 'at ' + authUsersdf['hourOfDay'].astype(str) + '-hour of weekday-' + authUsersdf['dayOfWeek'].astype(str) + ' ' + authUsersdf['sourceUser'] \
+ ' ' + authUsersdf['orientation'] + ' from ' + authUsersdf['sourceComputer'] + ' to ' + authUsersdf['destinationComputer'] + ' on ' + authUsersdf['authenticationType'] \
+ ' ' + authUsersdf['logonType'] + ' with ' + authUsersdf['successOrFailure']

In [ ]:
authUsersdf.head()

In [ ]:
authUsersdf.to_csv('/home/edo/Documents/Data/authSentences.txt', index=False)

The file authSentences.txt has all the authentications data labeled, and with one column for the sentence. Next step is the creation of train and test datasets.

In [ ]:
!gunzip /home/edo/Documents/Data/authSentences.txt.gz

In [ ]:
!head /home/edo/Documents/Data/authSentences.txt

## Create: authSentences >> authDocuments
Now from the sentences (i.e. events), we create the documents. For this purpose we group by each second and concatenate all the sentences that happen at the same time, separating them with the token [SEP].

In [1]:
import csv

In [2]:
import pandas as pd
labeledAuthdf = pd.read_csv('/home/edo/Documents/Data/authSentences.txt')
labeledAuthdf.head()

,Unnamed: 0,unixSecond,sourceUser,destinationUser,sourceComputer,destinationComputer,authenticationType,logonType,orientation,successOrFailure,hourOfDay,dayOfWeek,sentence
0,111,1,U101@DOM1,C1862$@DOM1,C1862,C1862,?,?,AuthMap,Success,1,1,at 1-hour of weekday-1 U101@DOM1 AuthMap from ...
1,112,1,U101@DOM1,U101@DOM1,C1862,C1862,Negotiate,Interactive,LogOn,Success,1,1,at 1-hour of weekday-1 U101@DOM1 LogOn from C1...
2,113,1,U10@DOM1,U10@DOM1,C229,C229,Kerberos,Network,LogOn,Success,1,1,at 1-hour of weekday-1 U10@DOM1 LogOn from C22...
3,114,1,U10@DOM1,U10@DOM1,C62,C528,Kerberos,Network,LogOn,Success,1,1,at 1-hour of weekday-1 U10@DOM1 LogOn from C62...
4,115,1,U1137@DOM1,U1137@DOM1,C1065,C1065,?,Network,LogOff,Success,1,1,at 1-hour of weekday-1 U1137@DOM1 LogOff from ...


Drop the columns that are not needed before creating the training and test datasets

In [3]:
labeledAuthdf.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)

In [4]:
labeledAuthdf.head()

,a,unixSecond,sourceUser,destinationUser,sourceComputer,destinationComputer,authenticationType,logonType,orientation,successOrFailure,hourOfDay,dayOfWeek,sentence
0,111,1,U101@DOM1,C1862$@DOM1,C1862,C1862,?,?,AuthMap,Success,1,1,at 1-hour of weekday-1 U101@DOM1 AuthMap from ...
1,112,1,U101@DOM1,U101@DOM1,C1862,C1862,Negotiate,Interactive,LogOn,Success,1,1,at 1-hour of weekday-1 U101@DOM1 LogOn from C1...
2,113,1,U10@DOM1,U10@DOM1,C229,C229,Kerberos,Network,LogOn,Success,1,1,at 1-hour of weekday-1 U10@DOM1 LogOn from C22...
3,114,1,U10@DOM1,U10@DOM1,C62,C528,Kerberos,Network,LogOn,Success,1,1,at 1-hour of weekday-1 U10@DOM1 LogOn from C62...
4,115,1,U1137@DOM1,U1137@DOM1,C1065,C1065,?,Network,LogOff,Success,1,1,at 1-hour of weekday-1 U1137@DOM1 LogOff from ...


In [5]:
labeledAuthdfSimple=labeledAuthdf.drop(columns=['a','sourceUser', 'destinationUser', 'sourceComputer', 'destinationComputer' \
                            , 'authenticationType','logonType','orientation','successOrFailure','hourOfDay','dayOfWeek'])

In [6]:
labeledAuthdfSimple.head()

,unixSecond,sentence
0,1,at 1-hour of weekday-1 U101@DOM1 AuthMap from ...
1,1,at 1-hour of weekday-1 U101@DOM1 LogOn from C1...
2,1,at 1-hour of weekday-1 U10@DOM1 LogOn from C22...
3,1,at 1-hour of weekday-1 U10@DOM1 LogOn from C62...
4,1,at 1-hour of weekday-1 U1137@DOM1 LogOff from ...


In [7]:
tempAuthDocument = labeledAuthdfSimple.groupby('unixSecond').apply(lambda x: '[SEP]'.join(x.sentence))

In [8]:
tempAuthDocument.head()

unixSecond
1    at 1-hour of weekday-1 U101@DOM1 AuthMap from ...
2    at 1-hour of weekday-1 U101@DOM1 TGS from C186...
3    at 1-hour of weekday-1 U10023@C14736 LogOn fro...
4    at 1-hour of weekday-1 U101@DOM1 TGS from C186...
5    at 1-hour of weekday-1 U10@DOM1 LogOff from C2...
dtype: object

In [9]:
tempAuthDocument.to_csv('/home/edo/Documents/Data/authDocuments.txt', index=True, quoting=csv.QUOTE_NONE, escapechar="\\")

Now we proceed to the creation of the training and testing datasets.

In [ ]:
import pandas as pd
authDocument = pd.read_csv('/home/edo/Documents/Data/authDocuments.txt')
authDocument.head()

In [17]:
authDocument=tempAuthDocument.to_frame()
authDocument.head()

,0
unixSecond,
1,at 1-hour of weekday-1 U101@DOM1 AuthMap from ...
2,at 1-hour of weekday-1 U101@DOM1 TGS from C186...
3,at 1-hour of weekday-1 U10023@C14736 LogOn fro...
4,at 1-hour of weekday-1 U101@DOM1 TGS from C186...
5,at 1-hour of weekday-1 U10@DOM1 LogOff from C2...


In [18]:
redteamdf = pd.read_csv('/home/edo/Documents/Data/redteam.txt',header=None)

In [19]:
redteamdf=redteamdf.rename(columns={0:'unixSecond', 1:'sourceUser', 2:'sourceComputer', 3:'destinationComputer'})
redteamdf['threatOrNormal']='threat'
redteamdf.head()

,unixSecond,sourceUser,sourceComputer,destinationComputer,threatOrNormal
0,150885,U620@DOM1,C17693,C1003,threat
1,151036,U748@DOM1,C17693,C305,threat
2,151648,U748@DOM1,C17693,C728,threat
3,151993,U6115@DOM1,C17693,C1173,threat
4,153792,U636@DOM1,C17693,C294,threat


In [23]:
labeledAuthdf=pd.merge(authDocument,redteamdf, on=['unixSecond'], how='left')

KeyError: ('unixSecond', 'threatOrNormal')

In [25]:
labeledAuthdf=labeledAuthdf.fillna('normal')

In [30]:
del labeledAuthdf['sourceUser']

KeyError: 'sourceUser'

In [32]:
del labeledAuthdf['sourceComputer']
del labeledAuthdf['destinationComputer']

In [33]:
labeledAuthdf.head()

,unixSecond,0,threatOrNormal
0,1,at 1-hour of weekday-1 U101@DOM1 AuthMap from ...,normal
1,2,at 1-hour of weekday-1 U101@DOM1 TGS from C186...,normal
2,3,at 1-hour of weekday-1 U10023@C14736 LogOn fro...,normal
3,4,at 1-hour of weekday-1 U101@DOM1 TGS from C186...,normal
4,5,at 1-hour of weekday-1 U10@DOM1 LogOff from C2...,normal


In [34]:
import numpy as np
from sklearn.model_selection import train_test_split

In [48]:
Y = labeledAuthdf['threatOrNormal']
X_train, Y_test = train_test_split(labeledAuthdf, Y , test_size=0.20, random_state=42, stratify=Y)

ValueError: too many values to unpack (expected 2)

In [46]:
X_train[0].count()

unixSecond        4008988
0                 4008988
threatOrNormal    4008988
dtype: int64

In [ ]:
print(X_train.count())
print(X_test.count())

In [ ]:
filter_list=[3,5]

In [ ]:
authDocument[authDocument.index.isin(filter_list)]

In [ ]:
print(row_list[4])

In [ ]:
common = authDocument.merge(redteamdf,on=['col1','col2'])